In [ ]:
pip install python-chess==0.28.1

In [ ]:
conda install pandas

In [ ]:
from IPython.core.display import HTML, display
HTML("""
<style>
svg {
    width:40% !important;
    height:40% !important;
}

.container { 
    width:100% !important;
}
</style>
""")

# Benutzerhandbuch


## Bedienen des Programms

Beim Spielstart wird zunächst das aktuelle Schachbrett in der Konsole ausgegeben. 
Ist der Spieler, der an der Reihe ist, ein menschlicher Spieler, so wird zusätzlich eine Liste aller legalen Züge ausgegeben. Dann wird der Spieler zur Eingabe einer dieser Züge aufgefordert. Die Eingabe muss in den ersten zwei Zeichen das Feld enthalten, von dem eine Figur aus bewegt werden soll, und bei Zeichen drei und vier die Felder, auf die die Figur bewegt werden soll. Ist der eingegebene Zug möglich, so wird dieser auf dem Schachfeld durchgeführt und erneut ausgegeben. Dann ist der nächste Spieler am Zug. Andernfalls wird die Aufforderung zur Eingabe eines Zuges so lange wiederholt, bis eine legale Eingabe vorliegt und der Zug durchgeführt werden kann.

Sobald das Spiel zu Ende ist, wird der Sieger des Spiels genannt. Danach wird der Spieler gefragt, ob er mit den gleichen Einstellungen das Spiel nochmal wiederholen möchte. Mit Eingabe einer "1" und bestätigen durch die 'Enter' Taste kann dies bejaht werden. So wird das Spiel nochmal mit den gleichen Einstellungen wiederholt. Andernfalls wird das Programm beendet.

Jederzeit kann mit der Tastenkombination 'Strg' + 'C' das Programm und somit auch das Spiel vorläufig beendet werden.

# Zugfindung durch KI

Die zentrale Aufgabe der KI ist es, den best möglichen Zug für eine gegebene Situation zu finden. Die Findung des Zuges findet dann wie folgt statt:

In [ ]:
OPENING_MAX_FULLMOVE_NUM = 6
MAX_BOARD_VALUE = float("inf")
MAX_DEPTH = 8
TIME_LIMIT = 45

def get_move(board):        
        opening_book = import_opening_book(OPENING_BOOK_LOC)
        
        if board.fullmove_number <= OPENING_MAX_FULLMOVE_NUM:
            move = get_opening_move(board, opening_book)
            if not move is None:
                return move
        

        return iterative_deepening(board, MAX_DEPTH, evaluate_board)

Dabei wird zunächst das Opening Book importiert und, wenn der Zug noch in dem Opening Book enthalten sein kann, geprüft, ob dieser einen passenden Zug vorschlägt. ist dies der Fall, wird dieser Zug genommen. Andernfalls wird in die Funktion `iterative_deepening` weitergegeben, die den Zug über heuristische Methoden bewertet.

# Einbinden und Verwendung von Opening-Books

Im Fall, dass sich das Spiel noch im Anfangszustand befindet, kann ein Opening Book zur Hilfe genommen werden, um einen passenden Zug zu finden.

Opening-Books sind ein wesentlicher Bestandteil der Schach-KI, da diese aus einer großen Anzahl von Eröffnungszügen bestehen. Hierbei wurden die Opening-Books bereits analysiert und konnten durch Überprüfungen als gute Eröffnungen identifiziert werden.

Zur Verwendung dieser Bücher müssen diese Dateien, die im .bin-Format vorliegen, in eine Variable geladen werden, da später auf diese Bücher mittels Funktionen zugegriffen werden soll. Damit die Eröffnungszüge in einer Variable gespeichert werden können, muss der Pfad zu dem Buch vorliegen, der im folgenden Codeausschnitt in eine globale Konstante `OPENING_BOOK_LOC` geschrieben wurde. Dies findet ebenfalls in der Datei `player/ai.py`statt.

In [ ]:
OPENING_BOOK_LOC = "./res/polyglot/Performance.bin"

Für das Laden eines Buches in eine Variable wird die Funktion `import_opening_book(self, book_location)` genutzt. Diese hat die Eigenschaft, dass sie als Übergabeparameter eine Konstante erhält, die den Pfad zu dem zu importierenden Opening-Book enthält.

Innerhalb der `import_opening_book` Funktion wird mittels einer If-Verzweigung überprüft, ob der angegebene Pfad eine Datei ist. Sollte dies zutreffen, dann wird, inklusive des Buchpfades, die Funktion `polyglot.open_reader` aus der "chess" Bibliothek aufgerufen. Diese Funktion liefert als Rückgabewert das Opening-Book, welches an die Stelle zurückgegeben wird, an der die Funktion `import_opening_book` aufgerufen wird.

Sollte der Pfad in der Variable `book_location` keine Datei sein, so wird ein _File Not Found_-Fehler geworfen, der dabei den vermeintlichen Pfad mit übergibt.

In [ ]:
def import_opening_book(book_location):
        if os.path.isfile(book_location):
            return chess.polyglot.open_reader(book_location)
        else:
            raise FileNotFoundError(
                errno.ENOENT, os.strerror(errno.ENOENT), book_location)

Die Funktion zum Importieren des Opening-Books wird im Konstruktor der KI-Klasse aufgerufen, da das Opening-Book bei jeder Verwendung direkt zu Beginn von der KI benötigt wird und somit direkt verfügbar sein muss. Das bringt den Vorteil, dass eine Klassenvariable vorliegt, die von allen Funktionen der Klasse KI verwendet werden kann, sobald der Konstruktor ausgeführt wurde.

Als Übergabewert wird die zuvor definierte Konstante `OPENING_BOOK_LOC` übergeben, die den Pfad zu einem Opening-Book enthält.

Um das Opening-Book verwenden zu können und aus diesem mögliche Eröffnungsstrategien verwenden zu können wird die Funktion `get_opening_move(self, board, opening_book)` benötigt, die als Übergabewerte das aktuelle Schachbrett und das importierte Book erhält.

Der aktuelle Spielstand in Form des Schachbretts wird benötigt damit das Opening-Book weiß auf welche Situation reagiert werden muss.

Der Sinn der Funktion `get_opening_move` ist es einen Schachzug als Objekt _chess.Move_ zurückzugeben, der laut des Opening-Books in dieser Situation angebracht ist. Dafür wird zuerst überprüft, ob die übergebene Variable `opening_book` durch den Konstruktor und dem damit einhergehenden Aufruf der Funktion `import_opening_book` korrekt initialisiert wurde.

Sollte dies nicht der Fall sein wird von der Funktion ein `None` zurückgegeben. Der Wert `None` ist in der Sprache Python der leere Zustand. 

Falls das Opening-Book korrekt geladen werden konnte wird durch ein try-except versucht einen passenden Schachzug zu finden. Hierbei wird ein try-except verwendet, da die aufgerufene Funktion `opening_book.weighted_choice(board)` einen _Index Error_ wirft, falls das Opening-Book keinen passenden Zug kennt. Wenn dieser Fall eintreten sollte wird ebenfalls ein `None` zurückgegeben. Sollte jedoch das Buch einen passenden Zug haben, dann wird dieser Zug in eine Variable `main_entry` geladen und aus dieser der Zug extrahiert. Die Funktion `weighted_choice` wählt dabei aus den Zügen, die am besten zu dieser Situation passen, einen zufällig aus. Dies macht das Spiel weniger vorhersehbar, wenn der gleiche Gegner mehrfach gegen die KI spielt.

Nachdem dies durchgeführt wurde, muss das Opening-Book geschlossen werden, damit es bei der nächsten Verwendung korrekt genutzt werden kann. Ebenfalls wird der erfolgreich extrahierte Zug zurückgegeben.

In [ ]:
def get_opening_move(board, opening_book):
        '''
        get the current board and return move, as string, for this situation
        '''
        if not (opening_book is None):
            try:
                main_entry = opening_book.weighted_choice(board)
                move = main_entry.move
                opening_book.close()
                return move
            except IndexError:
                return None
        else:
            return None

# Implementierung des Iterative Deepening Algorithmus

Um den besten Zug zu finen muss eine Schach-KI im Optimalfall alle möglichen Züge bis zum Ende des Spiels durchgehen, diese bewerten und den aussichtsreichsten Zug wählen. Da dies aber technisch nicht realistisch ist, werden die Züge nur bis zu einer gewissen Tiefe angeschaut und die Zustände über mitgegebene Evaluierungsfunktionen bewertet.

Die KI geht dabei dann wie folgt vor: Alle möglichen Züge werden durchgegangen. Die daraus entstehenden Zustände werden analysiert und evaluiert. Dabei jedoch zählt nicht nur der direkt erreichbare Zustand , sondern auch die aus diesem Zustand erreichbaren Zustände und so weiter. Aus diesem Grund wird immer bis zu einer bestimmten Tiefe in die Züge hineingeschaut und die sich daraus ergebenden Zustände evaluiert.

Um dies jedoch nicht fest immer bis zu einer bestimmten Tiefe durchgehen zu lassen, sondern variabel anzupassen, je nachdem wie viele Züge von dem gegebenen Zustand aus möglich sind, kann ein Zeitlimit dienen. Dabei wird anfangs die Tiefe auf 1 gesetzt und dann mittels des Minimax-Algorithmus die Züge evaluiert. Danach wird die Tiefe um 1 erhöht und erneut der Minimax-Algorithmus angewandt. Dies wird solange wiederholt, bis die angegebene Zeit abgelaufen ist. Dieser Algorithmus nennt sich "Iterative Deepening".

Dem Algorithmus muss dazu der aktuelle Zustand sowie eine maximale Tiefe mitgegeben werden. Ist diese erreicht bricht der Algorithmus ab, unabhängig davon, ob das Zeitlimit überschritten ist oder nicht. Zunächst müssen beim Ausführen einige Werte festgelegt werden. Der folgende Algorithmus zeigt, wie der Prozess des Iterative Deepening in der `player/ai.py` implementiert ist.

In [ ]:
def iterative_deepening(board, max_depth, evaluation_func):
    depth = 1

    start_time = int(time.time())
    end_time = start_time + TIME_LIMIT
    current_time = start_time

    player = bool(board.turn)
    best_possible_result = get_best_possible_result(board, player)

    legal_moves = list(board.legal_moves)
    while current_time < end_time and depth <= max_depth:
        move_val_dict = {}

        best_value = float('-inf')
        best_move = legal_moves[0]

        for move in legal_moves:
            board.push(move)
            value = min_value(board, player, float('-inf'), float('inf'), depth - 1, end_time, evaluation_func, best_possible_result)
            board.pop()
            if value is False:
                value = float('-inf')
            move_val_dict[move] = value
            if value == MAX_BOARD_VALUE:
                return move
            if value > best_value:
                best_value = value
                best_move = move

        legal_moves.sort(key=move_val_dict.get, reverse=True)
        depth += 1
        current_time = int(time.time())

    return best_move

Zunächst wird die Starttiefe auf 1 festgesetzt. Danach wird die Startzeit auf die aktuelle Zeit gesetzt und die Endzeit berechnet, indem auf die Startzeit das Zeitlimit addiert wird. Zudem wird der erste Wert für die aktuelle Zeit auf die Startzeit festgelegt.

Anschließend wird für den Spieler, der aktuell am Zug ist, berechnet, was das bestmöglich zu erreichende Resultat ist. Dies wird mit der Funktion `get_best_possible_result` durchgeführt. Dies ist dazu gut, um finale Zustände dahingehend zu evaluieren, ob diese für den Nutzer die best mögliche Option ist (Sieg oder Unentschieden wenn Sieg nicht mehr möglich ist) und dementsprechend zu bewerten. Die Funktion sieht wie folgt aus.

In [ ]:
def get_best_possible_result(board, player):
    if player and board.has_insufficient_material(chess.WHITE):
        return 0
    if not player and board.has_insufficient_material(chess.BLACK):
        return 0
    if player and not board.has_insufficient_material(chess.WHITE):
        return 1
    if not player and not board.has_insufficient_material(chess.BLACK):
        return -1

Dabei muss der Funktion der aktuelle Zustand sowie der Spieler, der an der Reihe ist, mitgegeben werden. Ist der aktuelle Spieler der der weißen Figuren (player == True) und hat weiß unzureichende Materialien für einen Sieg, so ist der bestmögliche Zustand ein Patt. Das gleiche Ergebnis wird zurückgegeben, wenn der Spieler der der schwarzen Figuren ist (player == False) und schwarz unzureichende Materialien hat.

Ist der Spieler jedoch weiß und er hat noch ausreichend Materialien, so wird der Wert 1 zurückgegeben, da ein Sieg noch erreichbar ist. Genauso wird für den schwarzen Spieler der Wert -1 zurückgegeben, falls er noch ausreichende Materialien besitzt, da dieser noch einen Sieg erreichen kann und der Wert -1 für einen Sieg von Schwarz steht.

Nach dieser Abfrage wird der eigentliche Algorithmus des "Iterative Deepening" durchgeführt.

Dabei wird zunächst eine Liste aller legalen Züge erstellt. Dann wird eine Schleife so lange durchlaufen, bis entweder die Zeit abgelaufen ist oder aber die maximale Tiefe erreicht ist. 

In dieser Schleife wird ein Dictionary aller Züge mit ihren berechneten Werte erstellt. Zudem werden Anfangswerte für die besten Züge und dessen Wert festgelegt. Der Anfangswert des besten Zugs wird auf den ersten Zug festgesetzt. Der Wert dieses wird auf den Wert "- Unendlich" gesetzt.

Nun wird über alle legalen Züge iteriert und jeder Zug einmal ausgeführt. Nun wird mittels des Minimax-Algorithmus der Wert dieses boards ermittelt. Dabei wird als Alpha "- Unendlich" und als Beta "Unendlich" mitgegeben. Zudem wird die Tiefe auf einen Wert festgelegt, der um einen Wert geringer ist als die maximale Tiefe, da durch Aufruf der Funktion in die erste Tiefe hineingegangen wurde. Zudem wird die Zeit mitgegeben, zu der der Algorithmus enden muss, damit der Minimax Algorithmus dementsprechend endet und das Zeitlimit nicht überschreitet.

Nachdem der Minimax-Algorithmus fertig durchlaufen ist, wird der Zug zunächst wieder rückgängig gemacht, damit dieser nicht das Spiel beeinflusst, sollte er gar nicht gewählt werden. Zudem wird der Wert mit dem Zug zu dem Dictionary hinzugefügt. Gleicht der Wert dem maximalen Wert für einen Zustand, ist also dementsprechend ein Sieg, wird der Zug direkt zurückgegeben, da mit diesem dann auf jeden Fall ein Sieg erreich werdne kann. Andernfalls wird der Wert verglichen, ob er besser ist als der aktuelle Wert. Ist dies der Fall, so wird der neue beste Wert auf den aktuell berechneten festgelegt, ebenso wie der beste Zug auf den der aktuellen Iteration gesetzt wird.

Nachdem alle Züge durchlaufen wurden, wird die Liste aller legalen Züge an Hand der berechneten Werte sortiert, damit im nächsten Durchlauf die Züge in dieser Reihenfolge durchlaufen werden. Dies verbessert den Durchsatz beim Alpha Beta Pruning und garantiert zudem, dass der beste Wert der vergangenen Runde gewählt wird, falls der Minimax-Algorithmus beim Durchlaufen der nächst tieferen Tiefe das Zeitlimit erreicht, bevor die Runde komplett evaluiert werden konnte.

Abschließend wird noch die Tiefe um 1 erhöht und die aktuelle Zeit auf die Systemzeit gesetzt, damit an Hand dieser entschieden werden kann, ob der Algorithmus noch weiter durchlaufen darf.

Nachdem dann die Zeit abgelaufen ist und alle Züge in der für die angegebenen Zeit maximalen Tiefe evaluiert wurden, wird der best mögliche Zug zurückgegeben. Dieser wird dann von der KI ausgeführt.

# Implementierung des Minimax-Algorithmus mit Alpha-Beta-Pruning

Um den bestmöglichen Zug zu erkennen, wird beim Minimax Algorithmus jeder Zug bis zu einer gewissen Tiefe betrachtet. Dabei wird unter der Prämisse gehandelt, dass auch der Gegner stets den besten Zug macht. Dies führt dazu, dass der bestmöglichste Zug ausgewählt wird (max), der erreichbar ist, wenn der Gegner mit dem für ihn jeweils besten Zug antwortet, der für den Spieler somit der schlechteste ist (min).

Die Umsetzung dabei erfolgt in zwei Funktionen - `min_value` und `max_value`. Auch diese sind direkt in der Datei `player/ai.py` vorzufinden. Erstere berechnet dabei den schlecht möglichsten Ausgang für den Spieler aus einer bestimmten Position, also den besten Ausgang für den Gegner. Letztere berechnet den best möglichsten Ausgang. Beide Funktionen sind sehr ähnlich aufgebaut und in folgendem Code-Snippet zu sehen.

In [ ]:
def min_value(board, player, alpha, beta, depth, time_limit, evaluation_func, best_possible_result):
    v = float('inf')
    
    if board.is_game_over() or depth == 0:
        return evaluation_func(board, player, best_possible_result)
    if int(time.time()) >= time_limit:
        return False

    for move in board.legal_moves:
        board.push(move)
        deeper_val = max_value(board, player, alpha, beta, depth -1, time_limit, evaluation_func, best_possible_result)
        board.pop()
        if deeper_val is False:
            return False            
        v = min(v, deeper_val)  

        if v <= alpha:
            return v
        beta = min(beta, v)
    return v

Neben dem akteullen Zustand des Spiels und dem Spieler, für den es den Zustand zu evaluieren gilt, wird außerdem eine Tiefe sowie ein Zeitlimit mitgegeben sowie die Werte Alpha und Beta. Alpha und Beta sind dabei dazu da, um den Minimax-Algorithmus zu beschleunigen, indem nicht jeder mögliche Zustand betrachtet wird. Durch diese Werte fallen nämlich solche weg, die direkt als irrelevant betrachtet werden können, da ohnehin bereits ein besserrer (im Fall von `min_value`) bzw. schlechterer (im Fall von `max_value`) Wert gefunden wurde.

Zunächst wird überprüft, ob das Spiel bereits vorbei ist oder die mitgegebene Tiefe auf 0 liegt. In beiden Fällen wird der aktuelle Zustand direkt evaluiert und zurückgegeben. Ansonsten wird geprüft, ob das übergebene Zeitlimit bereits erreicht wurde. In dem Fall wird der maximal negative Wert zurück gegeben, damit dieser Zug keine weitere Beachtung mehr bei der endgültigen Auswahl findet.

Ist auch dies nicht der Fall, werden alle von dem gegebenen Zustand aus erreichbaren Zustände durchgegangen. Dazu wird der Zug temporär ausgeführt und später wiede rrückgängig gemacht. Dieser neue, temporäre Zustand wird dann in die nächst tiefere Iteration gegeben, bei der nun der maximale Wert gesucht wird. Dabei wird auch der Spieler mitgegeben sowie die Werte Alpha und Beta und das Zeitlimit ebenso wie die um eins reduzierte Tiefe. Ist dabei ein Wert dabei, der kleiner ist als das aktuelle v, wird dieser Wert als das neue v genommen. Andernfalls bleibt v beim akteullen Wert.

Anschließend wird überprüft, ob der Wert v kleiner ist als der aktuelle Alpha Wert. Ist dies der Fall, muss der Pfad keine weitere Beachtung finden und es kann direkt v zurück gegeben werden. ist dies nicht der Fall, so wird Beta auf v gesetzt, falls dieser Wert kleiner als das aktuelle Beta ist. Nach Durchgang aller legalen Züge wird dann der sich aus all diesen Iterationen ergebende Wert v zurück gegeben.

Die `max_value` Funktion läuft similar ab, mit dem einzigen Unterschied, dass hier der maximale statt der minmale Wert gesucht wird und dementsprechend die Vergleiche sowie Startwerte angepasst sind. Außerdem gibt dieser bei der um eins tieferen Iteration in die `min_value` Funktion, an der Stelle, an der diese in die `max_value` Funktion gibt. Der restliche Aufbau bleibt unverändert.

In [ ]:
def max_value(board, player, alpha, beta, depth, time_limit, evaluation_func, best_possible_result):
    v = float('-inf')

    if board.is_game_over() or depth == 0:
        return evaluation_func(board, player, best_possible_result)
    if int(time.time()) >= time_limit:
        return False

    for move in board.legal_moves:
        board.push(move)
        deeper_val = min_value(board, player, alpha, beta, depth -1, time_limit, evaluation_func, best_possible_result)
        board.pop()
        if deeper_val is False:
            return False
        v = max(v, deeper_val)

        if v >= beta:
            return v
        alpha = max(alpha, v)
    return v

Mit diesem Algorithmus wird ein Baum aus allen Pfaden erstellt, an Hand der der beste Zug ermittelt werden kann. Da es sich jedoch nur selten um Endzustände handelt, für die eine Bewertung trivial erfolgen kann, ist eine Evaluierung der Zustände nötig. Diese wird im nächsten Kapitel beschrieben.

# Evaluierung eines gegebenen Schachbretts

Um gegebene Zustände auch mitten im Spiel bewerten zu können, müssen diese an Hand bestimmter Kriterien bewertet werden können, die über Sieg oder Niederlage hinausgehen. Dazu gibt es verschiedene Ansätze. Einige davon wurden im Laufe des Projektes umgesetzt und implementiert. Diese werden in diesem Kapitel erläutert. Zunächst jedoch gilt es aufzuzeigen, wie die Evaluierung der Zustände im generellen umgesetzt wird.

Zunächst wird zum Evaluieren eines Zustandes jener Zustand sowie der Spieler, für den dieser zu evaluieren ist, in die Funktion `evaluate_board` gegeben, die die Evaluierung zentral verwaltet.

In [ ]:
def evaluate_board(board, player, best_possible_result):
    player_color = chess.WHITE if player else chess.BLACK

    if board.is_game_over():
        result = get_board_result(board)
        if result is best_possible_result:
            return MAX_BOARD_VALUE
        if result is best_possible_result * -1:
            return -1 * MAX_BOARD_VALUE
    
    return get_board_value(board, player)

Dabei wird zunächst an Hand des Spielers die Farbe dieses ermittelt, die später bei den einzelnen Evaluierungsfunktionen benötigt wird.

Dann wird für den Fall, dass der gegebene Zustand einem Endzustand gleicht, das Ergebnis dieses ermittelt. Gleicht dieses dem bestmöglichen Ergebnis, das mittels der `get_board_result` Funktion zuvor ermittelt wurde, so wird der maximale Wert (Unendlich) für den Zusand zurückgegeben. Ist das Ergebnis des übermittelten Zustands jedoch eine Niederlage, so wird der maximale Wert umgekehrt und zurückgegeben (minus Unendlich). 

Gleicht der Zustand keinem Endzustand, so wird die Funktion `get_board_value` aufgerufen, die den aktuellen Zustand beweret. 

## Materialbewertung

Bei der Bewertung aller vorhandenen Materialien auf dem Spielfeld werden alle Figuren der jeweiligen Spieler zusammengezählt und je nach Figur mit einem Wert multipliziert. 

Dabei ist zunächst jedem Figurentyp ein Wert zuzuweisen. Üblicherweise werden Bauern dabei 1 Punkt, Türmen 5 Punkte, Springern sowie Läufern jeweils 3 Punkte und der Dame 9 Punkte zugeordnet. Dies geschieht über die Funktion `assign_piece_value()`. Dabei wird der Typ angegeben und die Punkte zurückgegeben. Zusätzlich kann angegeben werden, ob auch der König einen Wert zugewiesen bekommen soll. Diese werden dann über die `map` Funktion nach Farbe in der `get_value_by_color` Funktion für alle auf den Feldern befindlichen Figuren zusammengerechnet.

In [ ]:
PAWN_VALUE = 1
ROOK_VALUE = 5
KNIGHT_VALUE = 3
BISHOP_VALUE = 3
QUEEN_VALUE = 9
KING_VALUE = 15

def assign_piece_value(piece_type, count_king=True):
    return {
        1: PAWN_VALUE,
        2: KNIGHT_VALUE,
        3: BISHOP_VALUE,
        4: ROOK_VALUE,
        5: QUEEN_VALUE,
        6: KING_VALUE if count_king else 0
    }.get(piece_type, 0)


def get_value_by_color(board, color, count_king=True):
    pieces_value = map(
        lambda piece_type: len(board.pieces(piece_type, color)) * assign_piece_value(piece_type, count_king), chess.PIECE_TYPES)
    return sum(pieces_value)


def get_board_value(board, color, count_king=True):
    white_value = get_value_by_color(board, chess.WHITE, count_king)
    black_value = get_value_by_color(board, chess.BLACK, count_king)

    return white_value - black_value if color is chess.WHITE else black_value - white_value


Um den Gesamtwert des Schachbretts zu berechnen muss zunächst der Wert aller weißer Figuren berechnet werden und von diesem der Wert aller schwarzen Figuren abgezogen werden. Je nach angegebenen Spieler wird für diesen ein positiver Wert zurückgegeben, wenn das Spiel zu dessen Gunsten verläuft und ein negativer, wenn dies nicht der Fall ist.

Damit die Werte der Spieler berechnet werden können, wird die Anzahl aller Figurentypen der jeweiligen Farbe berechnet und diese mit dem Wert der Figurentypen multipliziert. Am Ende werden die Ergebnisse für alle Figurentypen zusammengezählt und zurückgegeben.

# Starten des Spiels im Jupyter Notebook

Um das Spiel auch im Jupyter Notebook starten zu können ist hier eine
vereinfachte Variante des \texttt{chess\_master} vorzufinden. Dabei
werden keine Spieler dynamisch erstellt, sondern abwechselnd der Nutzer und die KI nach einem Zug gefragt.

Zum Ausgeben des aktuellen Zustands ist hier außerdem noch eine Funktion zu sehen, mittels der die Schachbretter als SVG im Jupyter Notebook ausgegeben werden können.

In [ ]:
from IPython.display import SVG, display

def print_board_svg(board):
    display(SVG(chess.svg.board(board=board)))

Um im Nachhinein das Ergebnis ausgeben zu können, muss dies noch ermittelt werden, indem je nach Zustand das passende Ergebnis zurückgegeben wird:

In [ ]:
def get_board_result(board):
    if board.is_variant_loss():
        return -1 if board.turn == chess.WHITE else 1
    elif board.is_variant_win():
        return 1 if board.turn == chess.WHITE else -1
    elif board.is_variant_draw():
        return 0

    if board.is_checkmate():
        return -1 if board.turn == chess.WHITE else 1
    if board.can_claim_draw():
        return 0
    if board.is_seventyfive_moves() or board.is_fivefold_repetition():
        return 0
    if board.is_insufficient_material():
        return 0
    if not any(board.generate_legal_moves()):
        return 0
    
    return 0

Um das Spiel zu starte, wird zunächst ein neues Schachbrett erzeugt und ausgegeben. Dann wird der Spieler nach seinem nächsten Zug gefragt und im nächsten Zug die KI aufgefordert einen zu berechnen. Danach wird der Zug zu dem board hinzugefügt und im nächsten Durchlauf wieder ausgegeben. Dies wird solange wiederholt bis das Spiel vorüber ist. Dazu sind zunächst noch alle nötigen Importe für das Spiel aufzulisten.

In [ ]:
import pandas as pd
import chess
import chess.svg
import chess.polyglot
import os
import numpy as np
import time

board = chess.Board()
while not board.is_game_over():
    print_board_svg(board)
    if board.turn:
        legal_moves = list(map(board.uci, board.legal_moves))
        move = None
        while move not in legal_moves:
            print("Legal moves:")
            print(legal_moves)
            move = input("Please enter your move: ")
        move = chess.Move.from_uci(move)
    else:
        move = get_move(board)

    board.push(move)

result = get_board_result(board)
if result is 1:
    print("{} (White) has won".format(players[0].name))
elif result is -1:
    print("{} (Black) has won".format(players[1].name))
else:
    print("Draw")
    